In [ ]:
!pip install streamlit

In [3]:
import streamlit as st
import os

if not os.path.exists("my_chroma_index"):
    import zipfile
    with zipfile.ZipFile("my_chroma_index.zip", "r") as zip_ref:
        zip_ref.extractall("my_chroma_index")

In [ ]:
!pip install streamlit llama-index-core llama-index-llms-openai llama-index-embeddings-huggingface chromadb
!pip install llama-index-vector-stores-chroma

In [ ]:
import torch
print(torch.__version__)


In [2]:
!pip install --upgrade torch torchvision torchaudio


   ---------------------------------------- 0.0/212.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/212.4 MB ? eta -:--:--
   ---------------------------------------- 0.3/212.4 MB ? eta -:--:--
   ---------------------------------------- 0.5/212.4 MB 1.0 MB/s eta 0:03:23
   ---------------------------------------- 0.5/212.4 MB 1.0 MB/s eta 0:03:23
   ---------------------------------------- 0.8/212.4 MB 958.5 kB/s eta 0:03:41
   ---------------------------------------- 1.0/212.4 MB 915.5 kB/s eta 0:03:51
   ---------------------------------------- 1.0/212.4 MB 915.5 kB/s eta 0:03:51
   ---------------------------------------- 1.3/212.4 MB 907.1 kB/s eta 0:03:53
   ---------------------------------------- 1.6/212.4 MB 912.1 kB/s eta 0:03:52
   ---------------------------------------- 1.8/212.4 MB 932.3 kB/s eta 0:03:46
   ---------------------------------------- 1.8/212.4 MB 932.3 kB/s eta 0:03:46
   ---------------------------------------- 2.1/212.4 MB 889.9 kB/s eta

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Mahdimtd\\anaconda3\\envs\\my_env\\Lib\\site-packages\\torch\\lib\\uv.dll'
Consider using the `--user` option or check the permissions.




  Moving to c:\users\mahdimtd\anaconda3\envs\my_env\lib\site-packages\torch\distributed\rpc\server_process_global_profiler.py
   from C:\Users\Mahdimtd\AppData\Local\Temp\pip-uninstall-l90l1_s8\distributed\rpc\server_process_global_profiler.py
  Moving to c:\users\mahdimtd\anaconda3\envs\my_env\lib\site-packages\torch\distributed\run.py
   from C:\Users\Mahdimtd\AppData\Local\Temp\pip-uninstall-l90l1_s8\distributed\run.py
  Moving to c:\users\mahdimtd\anaconda3\envs\my_env\lib\site-packages\torch\distributed\tensor\__init__.py
   from C:\Users\Mahdimtd\AppData\Local\Temp\pip-uninstall-l90l1_s8\distributed\tensor\__init__.py
  Moving to c:\users\mahdimtd\anaconda3\envs\my_env\lib\site-packages\torch\distributed\tensor\__pycache__\__init__.cpython-39.pyc
   from C:\Users\Mahdimtd\AppData\Local\Temp\pip-uninstall-l90l1_s8\distributed\tensor\__pycache__\__init__.cpython-39.pyc
  Moving to c:\users\mahdimtd\anaconda3\envs\my_env\lib\site-packages\torch\distributed\tensor\__pycache__\_api.c

In [ ]:
# app.py
import streamlit as st
import os

# ----------- فقط اولین بار ایندکس را unzip کن -----------
if not os.path.exists("my_chroma_index"):
    import zipfile
    with zipfile.ZipFile("my_chroma_index.zip", "r") as zip_ref:
        zip_ref.extractall("my_chroma_index")

# ----------- ایمپورت‌ها و لود ایندکس -----------
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext, load_index_from_storage
import chromadb
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openai import OpenAI

PERSIST_DIRECTORY = "my_chroma_index"
COLLECTION_NAME = "my_collection"
EMBED_MODEL_NAME = "PartAI/Tooka-SBERT-V2-Large"

# ----------- لود ایندکس و مدل Embedding -----------
@st.cache_resource(show_spinner=False)
def load_index_and_embed():
    chroma_client = chromadb.PersistentClient(path=PERSIST_DIRECTORY)
    chroma_collection = chroma_client.get_or_create_collection(name=COLLECTION_NAME)
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    index = load_index_from_storage(storage_context)
    embed_model = HuggingFaceEmbedding(model_name=EMBED_MODEL_NAME)
    return index, embed_model

index, embed_model = load_index_and_embed()

# ----------- پارامترهای OpenRouter و System Prompt -----------
OPENROUTER_API_KEY = "sk-..."  # کلید خود را وارد کن
OPENROUTER_MODEL = "mistralai/mistral-7b-instruct"

SYSTEM_PROMPT = """
شما یک دستیار هوشمند فارسی هستید. پاسخ خود را فقط و فقط بر اساس اسناد ارائه‌شده از ایندکس بده.
اگر جواب دقیق در اسناد نبود، بنویسید: «اطلاعات کافی وجود ندارد.»
پاسخ‌ها باید مختصر، علمی و بدون اضافه‌گویی باشند.
"""

llm = OpenAI(
    api_key=OPENROUTER_API_KEY,
    base_url="https://openrouter.ai/api/v1",
    model=OPENROUTER_MODEL,
    is_chat_model=True,
    system_prompt=SYSTEM_PROMPT,
)

# ----------- UI Streamlit -----------
st.set_page_config(page_title="دستیار هوشمند متنی", page_icon="📚")
st.title("📚 پرسش و پاسخ بر اساس ایندکس متنی")

question = st.text_input("سوال خود را وارد کنید:")

if question:
    with st.spinner("در حال پردازش..."):
        query_engine = index.as_query_engine(embed_model=embed_model, llm=llm)
        response = query_engine.query(question)
        st.markdown("#### پاسخ:")
        st.write(str(response))
